#E-Commerce Customer Cohort and Retention Analysis

#### **Objective:** To analyze customer behavior over time by grouping them into monthly cohorts to identify trends in user retention and churn.

##### **Tech Stack:** PySpark (Data Cleaning), Spark SQL (Transformation), Matplotlib/Seaborn (Visualization).

##### **Business Impact:** Identifying which months have the highest churn allows marketing teams to target specific user segments for re-engagement.

## Step 1: Load the Raw Sales Data into a Spark DataFrame
### What this step does:
We load the CSV file containing all sales transactions into a Spark DataFrame.

header=True tells Spark that the first row contains column names.

inferSchema=True automatically detects the correct data types (string, integer, double, etc.).

Once loaded, display(df) lets us preview the data and confirm that it loaded correctly.

In [0]:
df = spark.read.csv(
    "/Volumes/workspace/default/cohort_data/sales.csv",
    header=True,
    inferSchema=True
)

display(df)


## Step 2: Inspect the Column Names
### What this step does:
This prints out all the column names from the DataFrame.
It helps you understand the dataset structure, check for messy or unexpected column names, and identify which fields you’ll need to rename or clean before doing any analysis.

printSchema() wull print out all the column names and their data types.

In [0]:
print(df.columns)
df.printSchema()


## Step 3: Parse order_date into a Real Date (Handle Mixed Formats)
### What this step does:
Convert order_date from messy strings (e.g., 1/10/2020, 13-11-2020) into a proper date so Spark can group by month and compute cohorts without errors.

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, date_trunc

df_active = (
    df
    # renames (safe even if not present; but later columns must match)
    .withColumnRenamed("cust_id", "customer_id")
    .withColumnRenamed("qty_ordered", "quantity")
    .withColumnRenamed("price", "total_sales")
    .withColumnRenamed("Customer Since", "customer_since")

    # normalize raw order_date
    .withColumn("order_raw", col("order_date").cast("string"))
    .withColumn("order_raw", F.regexp_replace("order_raw", r"\s+", ""))      # remove spaces
    .withColumn("order_raw", F.regexp_replace("order_raw", r"[./]", "-"))    # turn / or . into -

    # parse using ONLY patterns (no try_to_date)
    .withColumn(
        "order_date",
        F.coalesce(
            F.to_date("order_raw", "d-M-yyyy"),    # 13-11-2020
            F.to_date("order_raw", "M-d-yyyy"),    # 1-10-2020
            F.to_date("order_raw", "d-MMM-yyyy"),  # 13-Nov-2020
            F.to_date("order_raw", "yyyy-M-d"),    # 2020-11-13
            F.to_date("order_raw", "yyyy-MM-dd")   # 2020-11-13
        )
    )

    .filter(F.lower(col("status")).isin("complete", "received"))
    .withColumn("order_month", F.to_date(date_trunc("month", col("order_date"))))
    .drop("order_raw")
)

display(df_active.select("customer_id", "order_date", "order_month", "status").limit(10))


## Step 4: Identify Each Customer’s First Purchase (Cohort Assignment)
### What this step does:
This step determines the first time each customer ever made a purchase, which is essential for cohort analysis.

We use this first purchase date to assign every customer to a cohort month, meaning the month they became a customer.

####Why this matters:
- Cohort analysis groups customers by when they first became active.
- This allows us to track retention behavior over time and compare how different cohorts perform.
####What we do:

- Group orders by customer_id
- Find the earliest order_date for each customer 
- Convert that date into a cohort_month by truncating it to the month level 
- Store cohort_month as a date to ensure consistency in later calculations
 
The resulting table contains one row per customer, representing the cohort they belong to.

In [0]:
from pyspark.sql.functions import min, date_trunc, col
from pyspark.sql import functions as F
from pyspark.sql.functions import col, min, date_trunc

# Find first order date per customer
first_purchase = (
    df_active.groupBy("customer_id")
    .agg(min("order_date").alias("first_order_date"))
    .withColumn("cohort_month", F.to_date(date_trunc("month", col("first_order_date"))))
)

# Assign cohort month
first_purchase = first_purchase.withColumn(
    "cohort_month",
    date_trunc("month", col("first_order_date"))
)

display(first_purchase.limit(10))

## Step 5: Combine Cohort Information With Orders and Calculate Months Since First Purchase
### What this step does:
This step attaches each customer’s cohort month (their first purchase month) back onto all of their later orders.
Then we calculate how many months have passed since the customer’s cohort start date for each order.
This is what allows us to track retention month‑by‑month.
#### Breakdown:

- We join df_active (all valid customer orders) with first_purchase (each customer’s first-ever order date and cohort month).
- After the join, every order row contains the customer’s cohort month.
- We then use months_between(order_month, cohort_month) to calculate how far along in the retention timeline the customer is:
                0 = first month (cohort month)
 
                1 = month after cohort
 
                2 = two months after cohort
                
                and so on…


This field (months_since_cohort) is the core of all retention calculations.

In [0]:
from pyspark.sql.functions import col, months_between

# Join active orders with first_purchase information
df_cohort = (
    df_active
        .join(first_purchase, on="customer_id", how="inner")
)

# Calculate months since cohort
df_cohort = df_cohort.withColumn(
    "months_since_cohort",
    months_between(col("order_month"), col("cohort_month")).cast("int")
)

display(df_cohort.select(
    "customer_id",
    "order_month",
    "cohort_month",
    "months_since_cohort"
).limit(10))


## Step 6: Calculate Cohort Size (Number of Customers in Each Cohort)
### What this step does:
In this step, we determine how many unique customers belong to each cohort.
A “cohort” is defined by the month a customer made their first-ever purchase.
#### Breakdown:

- First, we select only customer_id and cohort_month and remove duplicates so that each customer appears once per cohort.
- Then we count how many distinct customers belong to each cohort_month.
- This gives us the baseline cohort size, which will act as the denominator when calculating retention rates later.

A larger cohort means more customers joined in that month; a smaller cohort means fewer people became new customers.


In [0]:
from pyspark.sql.functions import countDistinct

# One row per customer per cohort
customers_per_cohort = (
    df_cohort
        .select("customer_id", "cohort_month")
        .dropDuplicates()
)

# Cohort size table
cohort_size = (
    customers_per_cohort
        .groupBy("cohort_month")
        .agg(countDistinct("customer_id").alias("cohort_size"))
        .orderBy("cohort_month")
)

display(cohort_size.limit(20))

## Step 7: Count Active Customers by Cohort and Month Index
### What this step does:
For each cohort and each month after a customer’s first purchase, we count how many distinct customers made a purchase. This gives us the numerator for retention (i.e., “how many came back in month N?”). We first ensure one record per customer per month, then aggregate to get active_customers for every (cohort_month, months_since_cohort) pair.


In [0]:
from pyspark.sql.functions import countDistinct

# Count distinct customers active in each month since their cohort month
active_by_month = (
    df_cohort
        .select("customer_id", "cohort_month", "months_since_cohort")
        .dropDuplicates()
        .groupBy("cohort_month", "months_since_cohort")
        .agg(countDistinct("customer_id").alias("active_customers"))
        .orderBy("cohort_month", "months_since_cohort")
)

display(active_by_month.limit(20))

## Step 8: Calculate Retention Rate for Each Cohort and Month
### What this step does:
Now that we have:

cohort_size → how many customers started in each cohort (denominator)

active_by_month → how many returned in each month (numerator)

We join these two tables and compute the retention rate:
retention_rate = active_customers / cohort_size

This gives us the percentage of customers who came back in each month since their first purchase.
Month 0 should always be 1.0 (100%) because every customer is “active” in their first month.

In [0]:
from pyspark.sql.functions import col

# Join cohort sizes with active customers
retention = (
    active_by_month
        .join(cohort_size, on="cohort_month", how="inner")
        .withColumn(
            "retention_rate",
            col("active_customers") / col("cohort_size")
        )
        .orderBy("cohort_month", "months_since_cohort")
)

display(retention.limit(20))

## Step 9: Validate Cohort and Order Month Distributions
### What this step does:
This step helps us double‑check that our cohort creation and monthly order grouping worked correctly. Before calculating retention or building visualizations, we quickly inspect:

 1. How many customers are in each cohort month
This confirms whether customers are spread across multiple cohorts or concentrated in just a few months.

This confirms whether customers are spread across multiple cohorts or concentrated in just a few months.

####How many customers per cohort_month?
   display
   (    
    first_purchase.groupBy("cohort_month").count().orderBy("cohort_month")
  )

2. How many completed orders happened in each order month
This shows the volume of active orders per month, helping us understand customer activity and seasonality.

####How many active orders per order_month?
   
   display
   (    
    df_active.groupBy("order_month").count().orderBy("order_month")
   )

Why this matters:
It ensures that your cohorts look sensible (e.g., not all in one month) and that your monthly order counts align with expected customer behavior. This verification step is helpful before computing retention curves or heatmaps.


In [0]:
# How many customers per cohort_month?
display(
    first_purchase.groupBy("cohort_month").count().orderBy("cohort_month")
)

# How many active orders per order_month?
display(
    df_active.groupBy("order_month").count().orderBy("order_month")
)

## Step 10: Create a Retention Matrix (Pivot for Heatmap)
### What this step does:
We limit the analysis to the first 0–12 months after a customer’s first purchase, then pivot the long retention table so each row is a cohort_month and each column is months_since_cohort (0…12). The cell values are the retention rates. This matrix format is ideal for a heatmap visualization, letting you quickly spot drop‑offs and strong‑retention cohorts across months.

In [0]:
from pyspark.sql import functions as F

# Keep a 0–12 month horizon (common for retention)
retention_12 = retention.filter(F.col("months_since_cohort").between(0, 12))

# Pivot to matrix form
retention_matrix = (
    retention_12
      .groupBy("cohort_month")
      .pivot("months_since_cohort", list(range(0, 13)))
      .agg(F.first("retention_rate"))
      .orderBy("cohort_month")
)

display(retention_matrix)  # In Plot options, choose Heatmap (X: months_since, Y: cohort_month)

## Step 11.a: Plot Monthly Retention Curves (One Line per Cohort)
### What this step does:
We visualize how each cohort’s retention changes over time (months 0–12). We first keep only the first 13 months, convert the Spark DataFrame to a small pandas DataFrame for quick plotting, and then draw a simple line chart where:

- X‑axis = months_since_cohort (0, 1, 2, …, 12)
- Y‑axis = retention_rate expressed as a percentage
- One line per cohort = makes it easy to compare decay patterns across different start months

This plot helps you quickly spot common drop‑off points (e.g., Month‑1 cliff) and identify which cohorts retain better or worse than others.


In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

# Convert your retention_matrix to pandas
plot_df = retention_matrix.toPandas().set_index('cohort_month')

plt.figure(figsize=(16, 10))
sns.heatmap(plot_df, annot=True, fmt=".0%", cmap="YlGnBu", cbar_kws={'label': 'Retention Rate'})
plt.title('Monthly Cohort Retention Rate (%)')
plt.ylabel('Cohort Month')
plt.xlabel('Months Since First Purchase')
plt.show()

## Step 11.b: Forming Matplotlib line chart


In [0]:
from pyspark.sql.functions import col

import matplotlib.pyplot as plt



# Keep a 0–12 month horizon

ret_curves = retention.filter(col("months_since_cohort").between(0, 12)) 



# Convert to pandas for quick plotting

pdf = (ret_curves

       .orderBy("cohort_month", "months_since_cohort")

       .toPandas())



# Basic line chart: one line per cohort

plt.figure(figsize=(10, 6))

for cohort, grp in pdf.groupby("cohort_month"):

    plt.plot(grp["months_since_cohort"], grp["retention_rate"] * 100,

             label=str(cohort)[:7], alpha=0.85)

plt.title("Cohort Retention Curves (Monthly)")

plt.xlabel("Months since cohort")

plt.ylabel("Retention (%)")

plt.grid(alpha=0.25)

plt.legend(ncol=2, fontsize=8)

plt.show()

## Step 12: Create KPI Table for Key Retention Milestones (M1, M3, M6, M12)
### What this step does:
This step builds a compact KPI table that summarizes retention at the most important lifecycle points:

- Month 1 (M1): Early retention — shows immediate drop‑off
- Month 3 (M3): Medium‑term engagement
- Month 6 (M6): Long‑term loyalty
- Month 12 (M12): One‑year retention

These KPIs help business teams quickly understand how well each cohort is performing without needing to look at the full heatmap or line chart.

#### How it works:

- Filter retention data to keep only months 1, 3, 6, and 12.
- Pivot the numbers so each cohort becomes one row with four columns (M1, M3, M6, M12).
- Rename the columns for readability.
- Order results by cohort_month.
- Display the final KPI table.

In [0]:
from pyspark.sql import functions as F

# Keep only the milestone months
kpis = (
    retention
      .filter(F.col("months_since_cohort").isin(1, 3, 6, 12))
      .groupBy("cohort_month")
      .pivot("months_since_cohort", [1, 3, 6, 12])
      .agg(F.first("retention_rate"))
      .withColumnRenamed("1",  "M1")
      .withColumnRenamed("3",  "M3")
      .withColumnRenamed("6",  "M6")
      .withColumnRenamed("12", "M12")
      .orderBy("cohort_month")
)

display(kpis)

## Step 13: Persist Outputs as Delta Tables (for Dashboards & Reuse)
### What this step does:
We save the final analysis tables to a managed Delta database so you can query them later from SQL dashboards (without rerunning the whole notebook) and safely continue work tomorrow.

- Creates a lightweight schema (database) named analytics if it doesn’t exist.
- Saves three key tables:

       analytics.retention_long → long format with cohort_month, months_since_cohort, retention_rate (perfect for heatmaps & line charts).

       analytics.retention_matrix → pivoted matrix (rows = cohort months, columns = months since).

       analytics.retention_kpis → milestone KPIs (M1, M3, M6, M12) per cohort.

These persist in the metastore so you can open Catalog → Workspace → analytics
 (or run SELECT * FROM WORKSPACE.
analytics.<table>) anytime, including in SQL Dashboard tiles.

In [0]:
# Create a lightweight schema to store our outputs
spark.sql("CREATE DATABASE IF NOT EXISTS analytics")

# Save three handy tables
retention.write.mode("overwrite").format("delta").saveAsTable("analytics.retention_long")
retention_matrix.write.mode("overwrite").format("delta").saveAsTable("analytics.retention_matrix")
kpis.write.mode("overwrite").format("delta").saveAsTable("analytics.retention_kpis")

## Step 14: Verify That Delta Tables Were Saved Correctly
### What this step does:
This final step confirms that all the Delta tables we saved (retention_long, retention_matrix, retention_kpis) are correctly stored in the analytics database and can be queried later—especially from SQL Dashboards.
It ensures your work is fully persistent and you can safely return tomorrow without re‑running your entire notebook.
#### Breakdown:

Show all databases to confirm that the analytics schema exists.
Show all tables inside analytics to ensure the three Delta tables were created.
Preview a few rows from each saved table to verify the content is correct and complete.
#### Why this matters:
These checks confirm that your results are now permanently stored in Databricks’ catalog, making them reusable for dashboards, future analysis, or continuation of the project without running the entire notebook again.

Storing as Delta Tables allows for downstream BI tools like PowerBI or Tableau to connect directly to the cleaned data.

In [0]:
# List databases (schemas)
spark.sql("SHOW DATABASES").show(truncate=False)

# List tables in the 'analytics' database
spark.sql("SHOW TABLES IN analytics").show(truncate=False)

# Preview a few rows from each table
display(spark.table("analytics.retention_long").limit(10))
display(spark.table("analytics.retention_matrix").limit(10))
display(spark.table("analytics.retention_kpis").limit(10))

## Step 15: Query the Retention Matrix for Dashboard Visualization (SQL Tile)
### What this step does:
This SQL query retrieves the full retention matrix table we saved earlier (analytics.retention_matrix).
It’s used in your SQL Dashboard tile to generate the cohort retention heatmap.

We sort the results by cohort_month so the heatmap shows cohorts in chronological order.

#### Why this matters:
The retention matrix is the core dataset behind your heatmap visualization.
Each row represents a cohort, and each column (0 to 12) represents months since first purchase.
This structured layout makes it easy for the dashboard to generate the heatmap.

In [0]:
%sql
SELECT *
FROM WORKSPACE.analytics.retention_matrix
ORDER BY cohort_month;

## My Key Takeaways from the Analysis: 
- The highest customer churn occurs within the first month, making early post-purchase engagement the most critical phase for improving retention.
- Customers who remain active beyond the first 2–3 months demonstrate stable, long-term behavior, indicating a clear transition from trial users to loyal customers.
- Month-1 retention is a strong predictor of long-term customer value, with cohorts performing well early continuing to retain users over time.
- Older acquisition cohorts consistently outperform newer cohorts in long-term retention, suggesting a decline in customer quality over time.
- Recent cohorts show higher acquisition volume but lower retention, highlighting a trade-off between growth and customer quality.
- Retention performance varies significantly across cohorts, reinforcing the importance of cohort-based analysis rather than relying on overall averages.
- A small subset of customers drives long-term engagement and value, emphasizing the need to protect and retain high-value users.
- Retention follows a predictable decay pattern, enabling more accurate forecasting of customer behavior and revenue.
- Certain cohorts consistently underperform across all time periods, indicating ineffective acquisition strategies or external factors such as seasonality.
- Cohort-based retention metrics provide a more accurate view of business health than raw user growth, supporting better strategic and investment decisions.